In [2]:
from DataCollector import Twitter
import pandas as pd
from tqdm.notebook import tqdm

from decouple import config

tweets = Twitter.Tweets(config('TWITTER_API_KEY'),
               config('TWITTER_API_SECRET'),
               config('TWITTER_ACCESS_TOKEN_KEY'),
               config('TWITTER_ACCESS_TOKEN_SECRET'))

for i in tqdm([0,1,2]):
    df = pd.read_csv('Data/FilteredTwitterIDs/' + str(i) + '.csv')
    ids = df.tweet_id.values.tolist()
    for j in [1,2,3,4]:
        for k in range(1,11):
            tweets.addTweets(ids[((j-1)*1000)+(k-1)*100:((j-1)*1000)+k*100])
        tweets.saveJSON('Data/Text/' + str(i) + str(j))
        tweets.reset()



  0%|          | 0/3 [00:00<?, ?it/s]